# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

## Dataset

### Overview

Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/truongvanthanh95/azure-machine-learning/refs/heads/main/loan_data.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone-Project-Experiment'

experiment=Experiment(ws, experiment_name)

In [4]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "auto-ml-compute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS3_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

In [5]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

automl_config = AutoMLConfig(
        task='classification',
        compute_target=trainCluster,
        training_data=ds,
        label_column_name='loan_status',
        n_cross_validations=5,
        **automl_settings
)

In [6]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment Id Type Status Details Page Docs Page Capstone-Project-Experiment AutoML_4d9924d8-8c60-4dbc-864b-a98f406900fd automl NotStarted Link to Azure Machine Learning studio Link to Documentation

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

2024-11-05 01:53:52.102955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 01:53:52.124025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 01:53:52.130468: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 01:53:52.336178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 01:53:53.361733: W tensorflow/comp

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment Id Type Status Details Page Docs Page Capstone-Project-Experiment AutoML_4d9924d8-8c60-4dbc-864b-a98f406900fd automl NotStarted Link to Azure Machine Learning studio Link to Documentation


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFea

{'runId': 'AutoML_4d9924d8-8c60-4dbc-864b-a98f406900fd',
 'target': 'auto-ml-compute',
 'status': 'Completed',
 'startTimeUtc': '2024-11-05T01:54:03.550813Z',
 'endTimeUtc': '2024-11-05T02:20:11.760556Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'auto-ml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"Capstone-Project-Experiment","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-269495","workspace_name":"quick-starts-ws-269495","region":"westeurope","compute_target"

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: Capstone-Project-Experiment,
Id: AutoML_4d9924d8-8c60-4dbc-864b-a98f406900fd_38,
Type: azureml.scriptrun,
Status: Completed)
precision_score_macro 0.9244271640553041
average_precision_score_macro 0.9692840197982523
matthews_correlation 0.8147697817881921
norm_macro_recall 0.7820729393239663
recall_score_micro 0.937711111111111
precision_score_micro 0.937711111111111
average_precision_score_weighted 0.9831624275092097
AUC_weighted 0.9802930798004013
recall_score_weighted 0.937711111111111
AUC_micro 0.9873396024691358
precision_score_weighted 0.9366909912592561
balanced_accuracy 0.8910364696619831
AUC_macro 0.9802930798004013
f1_score_macro 0.9062822677529414
average_precision_score_micro 0.9878222700562777
f1_score_weighted 0.9364406204358474
log_loss 0.14603373886236737
weighted_accuracy 0.9623362943878508
recall_score_macro 0.8910364696619831
accuracy 0.937711111111111
f1_score_micro 0.937711111111111
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_4d9924d8-8

In [9]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook269495/code/Users/odl_user_269495'}

prefittedsoftvotingclassifier
{'estimators': ['1', '30', '0', '29', '26', '21', '27'],
 'weights': [0.125, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'enable_categorical': False,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estim

In [10]:
#Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-269495', subscription_id='f5091c60-1c3c-430f-8d81-d802f6bf2414', resource_group='aml-quickstarts-269495'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.937711111111111'})


In [11]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

capstoneModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.937711111111111


HyperDrive_HighAccuracy version: 1
	 Accuracy : 0.8757037037037037
	 Regularization Strength : 4.0
	 Max Iterations : 200


